In [12]:
import pandas as pd
from sklearn import preprocessing

In [13]:
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)

In [14]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('Sources/test_final_100k.csv')
visitas = pd.read_csv('Sources/visitas_proc.csv')
preds = pd.DataFrame()

In [15]:
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(visitas, on=['idaviso','idpostulante'], how='left')
resultados = postulaciones['sepostulo']
postulaciones.drop(['titulo','descripcion','idaviso','idpostulante','sepostulo'],axis=1, inplace=True)
postulaciones.head()

,Unnamed: 0.1,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,Unnamed: 0,visito
0,177374,29.0,FEM,5,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,75.0,1.0,NaN,26.095588,NaN,NaN
1,447085,20.0,FEM,11,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,75.0,1.0,NaN,26.095588,NaN,NaN
2,145390,38.0,MASC,3,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,75.0,1.0,NaN,26.095588,NaN,NaN
3,460866,NaN,NO_DECLARA,-1,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,75.0,1.0,NaN,26.095588,NaN,NaN
4,358106,22.0,FEM,5,Part-time,Junior,Administración,ENRIQUE MEYRIALLE &amp; ASOC.,178.0,61.0,75.0,1.0,NaN,26.095588,NaN,NaN


In [ ]:
postulaciones.drop(['Unnamed: 0.1'],axis=1,inplace=True)

In [ ]:
visitas.head()

In [ ]:
test = test.merge(postulantes, on='idpostulante')
test = test.merge(avisos, on='idaviso')
test = test.merge(visitas, on=['idaviso','idpostulante'], how='left')
test.drop(['titulo','descripcion','idaviso','idpostulante','id'],axis=1, inplace=True)
test.head()

In [ ]:
test.drop(['Unnamed: 0.1'],axis=1,inplace=True)

In [6]:
test = pd.concat([test,postulaciones])

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not postulaciones[col].dtype == 'O': continue
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])

In [8]:
postulaciones = test[100000:].fillna(-1)
test = test[:100000].fillna(-1)

In [9]:
partitions = []
partitions_res = []
n = 5
index = int(postulaciones.shape[0]/n)
for i in range(n):
    partitions.append(postulaciones[index*i:index*(i+1)])
    partitions_res.append(resultados[index*i:index*(i+1)])


In [10]:
def proba(result):
    prob_1 = []
    for x in result:
        prob_1.append(x[1])
    return prob_1

In [11]:
print('Arranca XG')

aviso = None
postulantes = None
import xgboost as xgb
dtrain = xgb.DMatrix(data=partitions[0],label=partitions_res[0])
dtest = xgb.DMatrix(data=partitions[4])

print('Termino XG')

Arranca XG
Termino XG


In [13]:
param = {'max_depth':15, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
bst = xgb.train(param, dtrain, num_round)

IndexError: invalid index to scalar variable.

In [14]:
preds['xgb'] = bst.predict(dtest)

In [15]:
print('Arranca RF')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clf=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
clf.fit(partitions[1],partitions_res[1])
preds['RF'] = proba(clf.predict_proba(partitions[4]))

print('Termino RF')

Arranca RF
Termino RF


In [16]:
print('Arranca NB')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(partitions[2] ,partitions_res[2])
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
preds['NB'] = proba(nb.predict_proba(partitions[4]))

print('Termino NB')

Arranca NB
Termino NB


In [17]:
print('Arranca KNN')

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=25 ,n_jobs=-1, algorithm='auto')
neigh.fit(partitions[3], partitions_res[3])
preds['KNN'] = proba(neigh.predict_proba(partitions[4]))

print('Termino NB')

Arranca KNN
Termino NB


In [18]:
preds.to_csv('Sources/predicciones.csv')

In [19]:
print('Arranca PERCEPTRON')
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(preds, partitions_res[4])
#from sklearn import svm
#clf = svm.SVC()
#clf.fit(preds, partitions_res[4])

print('PERCEPTRON ya esta entrenado')

Arranca PERCEPTRON
PERCEPTRON ya esta entrenado


In [20]:
preds.head()

,xgb,RF,NB,KNN
0,0.108474,0.24,0.562943,0.08
1,0.730141,0.84,0.605802,0.60
2,0.989302,0.76,0.605802,0.40
3,0.325601,0.84,0.605802,0.64
4,0.798988,0.72,0.605802,0.72


In [21]:
print('Arranca XGB final')

aviso = None
postulantes = None
import xgboost as xgb
dtrain = xgb.DMatrix(data=postulaciones, label=resultados)
dtest = xgb.DMatrix(data=test)
param = {'max_depth':15, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
bst = xgb.train(param, dtrain, num_round)


Arranca XGB final


In [22]:
final = pd.DataFrame()

In [24]:
final['xgb'] = bst.predict(dtest)

print('Termino XGB final')

Termino XGB final


In [27]:
print('Arranca RF final')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clfr=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
clfr.fit(postulaciones,resultados)
final['RF'] = proba(clfr.predict_proba(test))

print('Termino RF final')

Arranca RF final
Termino RF final


In [26]:
test

,Unnamed: 0.1,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad
0,69266,42.0,1,21,1,1,30,456,45.0,-1.0,-1.0,-1.0,2,-1.0
1,83910,31.0,2,9,1,1,30,456,45.0,-1.0,-1.0,-1.0,2,-1.0
2,351736,36.0,1,12,1,1,30,456,45.0,-1.0,-1.0,-1.0,2,-1.0
3,15447,69.0,2,15,1,3,170,3864,214.0,-1.0,-1.0,-1.0,2,-1.0
4,138590,32.0,1,11,1,3,170,3864,214.0,-1.0,-1.0,-1.0,2,-1.0
5,99584,28.0,2,10,1,3,170,3864,214.0,-1.0,-1.0,-1.0,2,-1.0
6,304578,29.0,2,12,1,3,40,3036,123.0,-1.0,-1.0,-1.0,2,-1.0
7,206910,54.0,2,6,1,3,40,3036,123.0,-1.0,-1.0,-1.0,2,-1.0
8,340918,28.0,1,12,1,3,40,3036,123.0,-1.0,-1.0,-1.0,2,-1.0
9,70812,38.0,2,15,4,3,154,2466,113.0,-1.0,-1.0,-1.0,2,-1.0


In [ ]:
print('Arranca NB final')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(postulaciones, resultados)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
final['NB'] = proba(nb.predict_proba(test))

print('Termino NB final')

Arranca NB final


In [29]:
print('Arranca KNN final')

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=25 ,n_jobs=-1, algorithm='auto')
neigh.fit(postulaciones,resultados)
final['KNN'] = proba(neigh.predict_proba(test))

print('Termino KNN final')

Arranca KNN final
Termino KNN final


In [30]:
final.head()

,xgb,RF,NB,KNN
0,8.512726e-10,0.0,1.850918e-13,0.0
1,2.429953e-07,0.0,1.850918e-13,0.0
2,5.409165e-08,0.0,1.850918e-13,0.0
3,6.914397e-10,0.0,1.850918e-13,0.0
4,9.982956e-10,0.0,1.850918e-13,0.0


In [31]:
print('Predigo con AdaBoost')

resultado = clf.predict(final)

print('termino Adaboost')

Predigo con AdaBoost
termino Adaboost


In [32]:
resultado

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = resultado
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_pipe_fede.csv', index=False)

In [34]:
resultado[resultado == 1 ].shape

(58329,)